In [1]:
from bs4 import BeautifulSoup
import urllib
from selenium import webdriver
from time import sleep
import pandas as pd
import numpy as np

# Webscraping

## Get job_urls of all related search

In [2]:
job_urls = []
vPath = 'C:/Users/Arsyad/Downloads/chromedriver_win32/chromedriver'

In [3]:
driver = webdriver.Chrome(executable_path=vPath)

url_lst = ["https://www.mycareersfuture.sg/search?search=data%20analyst&page={}", #search data analyst
           "https://www.mycareersfuture.sg/search?search=data%20scientist&page={}", #search data scientist
           "https://www.mycareersfuture.sg/search?search=analytics&page={}", #search analytics
           "https://www.mycareersfuture.sg/search?search=machine%20learning&page={}", #machine learning
           "https://www.mycareersfuture.sg/search?search=business%20intelligence&page={}", #business intelligence
           "https://www.mycareersfuture.sg/search?search=business%20analyst&page={}", #business analyst
           "https://www.mycareersfuture.sg/search?search=data%20engineer&page={}", #data engineer
           "https://www.mycareersfuture.sg/search?search=artificial%20intelligence&page={}", #data engineer
           "https://www.mycareersfuture.sg/search?search=ai&page={}",
           "https://www.mycareersfuture.sg/search?search=data&page={}"
          ]
           
for u in url_lst:
    i = 0
    for i in range(0,30):
        driver.get(u.format(i))
        sleep(3) # Wait one second.
        html = BeautifulSoup(driver.page_source, 'lxml')
        for job_card in html.find_all('div', id=lambda x: x and x.startswith('job-card-')):
            job_urls.append('https://www.mycareersfuture.sg'+job_card.find('a')['href'])
      
    
driver.close()

In [4]:
len(job_urls)

2387

In [5]:
job_urls2 = set(job_urls)
len(job_urls2)

1550

In [6]:
#save urls as csv
pd.DataFrame(list(job_urls2)).to_csv('jobs_url_22jul.csv', encoding='utf-8')

In [7]:
job_df = pd.read_csv('jobs_url_22jul.csv')
job_ls = job_df.iloc[:,1].values

In [8]:
len(job_ls)

1550

## Loop through each job_url to get details

In [9]:
companies = []
titles = []
addresses = []
emp_type = []
seniority = []
category = []
salaries = []
responsibilities = []
requirements = []
failed_url = []

driver = webdriver.Chrome(executable_path=vPath)
for ind, url_job in enumerate(job_ls): #error at 1010, to use try except for future runs
    try:
        driver.get(url_job)
        sleep(5)
        html_job = driver.page_source
        soup_job = BeautifulSoup(html_job, 'lxml')
        companies.append(soup_job.find('p', {'id': 'company'}))
        titles.append(soup_job.find('h1', {'id': 'job_title'}))
        addresses.append(soup_job.find('p', {'id': 'address'}))    
        emp_type.append(soup_job.find('p', {'id': 'employment_type'}))
        seniority.append(soup_job.find('p', {'id': 'seniority'}))
        category.append(soup_job.find('p', {'id': 'job-categories'}))
        salaries.append(soup_job.find('div', {'class': 'lh-solid'}))
        responsibilities.append(soup_job.find('div',{'id':'job_description'}))
        requirements.append(soup_job.find('div',{'id':'requirements'}))
    except:
        failed_url.append(url_job)

driver.close()

In [10]:
#create dictionary and iterate through scraped list to append information
lists = [companies, titles, addresses, emp_type, seniority, category, salaries, responsibilities, requirements]
clean = {}
for ind, ls in enumerate(lists):
    clean[ind] = []
    for e in ls:
        try:
            clean[ind].append(e.text)
        except:
            clean[ind].append(np.nan)
#convert dictionary into dataframe and rename columns
df = pd.DataFrame(clean)
df.columns = ['Company', 'Title', 'Address', 'Emp_type', 'Seniority', 'Industry', 'Salary', 'Responsibility', 'Requirements']
df

,Company,Title,Address,Emp_type,Seniority,Industry,Salary,Responsibility,Requirements
0,NTT DATA SINGAPORE PTE. LTD.,Senior Application Developer,"KEPPEL TOWERS, 10 HOE CHIANG ROAD 089315",Contract,"Executive, Senior Executive",Information Technology,"$8,000to$110,000","Roles & Responsibilities Creating complex, en...",Requirements A Bachelor’s degree in Computer S...
1,HP PPS ASIA PACIFIC PTE. LTD.,Business Analyst,1 DEPOT CLOSE 109841,"Permanent, Full Time",Professional,Others,"$8,400to$12,900",Roles & ResponsibilitiesHP is the world’s lead...,RequirementsEducation and Experience Required:...
2,SHOPEE SINGAPORE PRIVATE LIMITED,Software Engineer,"GALAXIS, 1 FUSIONOPOLIS PLACE 138522","Permanent, Full Time",Executive,Information Technology,"$4,400to$8,000",Roles & ResponsibilitiesResponsibilities: De...,RequirementsRequirements: Minimum B.S. degre...
3,PRICEWATERHOUSECOOPERS CONSULTING (SINGAPORE) ...,Data Analytics –Manager,"MARINA ONE EAST TOWER, 7 STRAITS VIEW 018936","Permanent, Contract, Full Time",Manager,Consulting,"$6,200to$9,500",Roles & Responsibilities Advisory - Consulting...,Requirements A good Degree in a quantitative ...
4,Company Undisclosed,Research Fellow,NaN,"Contract, Full Time","Professional, Executive",Others,"$4,000to$5,000",Roles & ResponsibilitiesData Scientist / Progr...,Requirements Doctorate degree in a relevant fi...
5,SOFTENGER (SINGAPORE) PTE. LTD.,Data Center Operation (DC for - Server install...,"SIM LIM TOWER, 10 JALAN BESAR 208787",Contract,Professional,Information Technology,"$3,500to$4,000",Roles & Responsibilities Candidate should have...,RequirementsThe client is looking candidates w...
6,"JPMORGAN CHASE BANK, N.A.","Application Support Analyst, Reference Data Team",NaN,Full Time,Executive,Information Technology,"$6,000to$12,000",Roles & ResponsibilitiesWe are seeking an Appl...,RequirementsQualifications 7-10 years Product...
7,"JPMORGAN CHASE BANK, N.A.",CyberSecurity Business Resiliency Analyst and ...,NaN,Full Time,Executive,Information Technology,"$7,500to$15,000",Roles & ResponsibilitiesWorking in Cybersecuri...,Requirements• Bachelors’ degree in computer sc...
8,ADDSTONES SAS,Data Engineer,120 ROBINSON ROAD 068913,Permanent,Manager,Engineering,"$5,000to$10,000",Roles & ResponsibilitiesGFI is an internationa...,RequirementsDeep analytical skills : Busine...
9,BLUECHIP PLATFORMS ASIA PTE. LTD.,"Associate Director, Portfolio Analytics, Inves...",NaN,Full Time,Manager,Banking and Finance,NaN,Roles & Responsibilities Building of framework...,Requirements Minimum 6 years' relevant experie...


In [11]:
#save dataframe into csv
df.to_csv('jobs_df_22jul.csv', encoding='utf-8')